In [ ]:
from bs4 import BeautifulSoup as bs
import requests
import os
import pymongo
import time
from splinter import Browser
import pandas as pd
import re
import scrapy
    

In [ ]:
# use pandas to read in Kaggle csv dataset

data = pd.read_csv("train.csv") 
data.head

In [ ]:
#delete unneccesary columns

df = data.drop(columns=['user_name', 'review_description','region_2'])
df.head()

In [ ]:
# rename column that includes brand and wine title
df.rename(columns={'review_title':'brand_name'}, inplace=True)
df.head()

In [ ]:
# using splinter to browse target webpage

executable_path = {'executable_path': 'C:\\Users\\winyi\\OneDrive\\Desktop\\LearnPython\\chromedriver.exe'}

browser = Browser('chrome', **executable_path, headless=False)

url = "https://www.vivino.com/explore?e=eJwtzjEOwjAMBdDb_DltU0EHb9wAMSGETAhRJNIiJyrt7QlJFj_5fw8OQh2Cn6lTCLzRqBTMTpczTB4nfHLtXrSyeJv4jeVBwsnPLt55tcLOYqGnjQbfdL3l40JfGSq6cmidhvy749Acm1Oxr6vOf5R40ojk-AfdtDSE"
browser.visit(url)

time.sleep(5)





In [ ]:
# scraping the site
#create variables, add counter and while loop to scroll page

brand_list = []
name_list = []
rating_list = []
review_count_list = []
scroll_counter = 0
while scroll_counter < 125:
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);") 
    time.sleep(7)
    scroll_counter = scroll_counter+1

# beautiful soup and for loop to scrape info needed for eah wine on page    
    
html = browser.html    
soup = bs(html, 'lxml')       
wine = soup.find_all('div', {'class': 'explorerCard__explorerCard--3Q7_0'})    

for wine in wine:
    brand=wine.find('span', {'class':'vintageTitle__winery--2YoIr'})
    brand2 = brand.text
    brand_list.append(brand2)
    name= wine.find('span', {'class':'vintageTitle__wine--U7t9G'})
    name2=name.text
    name_list.append(name2)
    rating =wine.find('div',{'class':'vivinoRatingWide__averageValue--1zL_5'})
    rating2 = rating.text
    rating_list.append(rating2)
    review_count = wine.find('div', {'class':'vivinoRatingWide__basedOn--s6y0t'})
    review_count2 = review_count.text
    review_count_list.append(review_count2)
            
             
        

In [ ]:
# create dataframe from listsof data, re-name columns

vino_df = pd.DataFrame({'brand':brand_list,'name':name_list,'rating':rating_list,'review_count':review_count_list})

In [ ]:
#zip brand name and title of wine into one column to match kaggle data

vino_df['brand_name']=vino_df['brand']+' '+ vino_df['name']
vino_df

In [ ]:
# drop the two columns that have been merged
wine_df=vino_df.drop(columns=['brand', 'name'])
wine_df

In [ ]:
# merge two dataframes
clean_wine = pd.merge(df,wine_df,on='brand_name',how='left')
clean_wine.head()

In [ ]:
# export final dataframe to csv
clean_wine.to_csv('final_data.csv')


# clean_wine = clean_wine[clean_wine['rating'].notna()]
# clean_wine.head()

In [ ]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

db = client.wine_db

# Declare the collection
collection = db.wine_db

In [ ]:
clean_wine.reset_index(inplace=True)
clean_wine_dict = clean_wine.to_dict('records')

collection.insert_many(clean_wine_dict)


